# PFAS Data Request

### Request for Leupolds and Stevens

### Libraries Import

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### Data Import and Export

#### Define Functions

In [4]:
def read_excel_sheet(excel_file, sheet_name):
    """
    Read a specific sheet from an Excel file into a DataFrame.

    Args:
        excel_file (str): The path to the Excel file.
        sheet_name (str): The name of the sheet to read.

    Returns:
        pd.DataFrame or None: A DataFrame containing the sheet data or None if the sheet is not found.
    """
    try:
        # Load the specified sheet from the Excel file into a DataFrame
        df = pd.read_excel(excel_file, sheet_name=sheet_name, skiprows=1)
        return df
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

In [5]:
def dfs_to_excel(dfs, filename):
    """
    Write an array of dataframes to separate tabs in an Excel file.
    
    Parameters:
    - dfs: A dictionary where keys are the tab names and values are the DataFrames to write.
    - filename: The name of the Excel file to write to.
    """
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        for tab_name, df in dfs.items():
            # Write each dataframe to a separate sheet
            df.to_excel(writer, sheet_name=tab_name)
    
    print(f"File '{filename}' has been written with {len(dfs)} tabs.")

#### Import Data from Excel

In [6]:
# Example usage:

#Specify where the file is saved use '/' instead of '\'
excel_file_path = 'U:/Research_and_Innovation/_ModellingGroupProjects/PFAS Investigations/Data and analysis/Database as of 07-09-2024.xlsm'
#Specify sheet name
sheet_name = 'PFAS Database'

#Call data import function
data_df = read_excel_sheet(excel_file_path, sheet_name)

data_df.head()

,Sampling point Date Component,Sample Number,Sampling Point,Sample Name,Sampled Date,Analysis,Component,Qualifiers,Result,Units,...,ChainLength,Pseudonym,Dominant Contributor,Sector,Depth,Edit Notes,Field,Notes,Region,Crop
0,ANODIZE SOLUTIONS (FPT)-43692.3333333333-10:2F...,259452.0,ANODIZE SOLUTIONS (FPT),NaN,2019-08-15 08:00:00,PFAS-CL,10:2Fluorotelomersulfonic Acid (Report),NaN,<9.40,ppt_wt_v,...,12.0,Metal Finisher B,NaN,Metal Finishing,NaN,NaN,NaN,NaN,NaN,NaN
1,ANODIZE SOLUTIONS (FPT)-43692.3333333333-11Cl-...,259452.0,ANODIZE SOLUTIONS (FPT),NaN,2019-08-15 08:00:00,PFAS-CL,11Cl-PF3OUdS (Report),NaN,<4.70,ppt_wt_v,...,10.0,Metal Finisher B,NaN,Metal Finishing,NaN,NaN,NaN,NaN,NaN,NaN
2,ANODIZE SOLUTIONS (FPT)-43692.3333333333-4:2Fl...,259452.0,ANODIZE SOLUTIONS (FPT),NaN,2019-08-15 08:00:00,PFAS-CL,4:2Fluorotelomersulfonic Acid (Report),NaN,<4.70,ppt_wt_v,...,6.0,Metal Finisher B,NaN,Metal Finishing,NaN,NaN,NaN,NaN,NaN,NaN
3,ANODIZE SOLUTIONS (FPT)-43692.3333333333-6:2Fl...,259452.0,ANODIZE SOLUTIONS (FPT),NaN,2019-08-15 08:00:00,PFAS-CL,6:2Fluorotelomersulfonic Acid (Report),NaN,<19.0,ppt_wt_v,...,8.0,Metal Finisher B,NaN,Metal Finishing,NaN,NaN,NaN,NaN,NaN,NaN
4,ANODIZE SOLUTIONS (FPT)-43692.3333333333-8:2Fl...,259452.0,ANODIZE SOLUTIONS (FPT),NaN,2019-08-15 08:00:00,PFAS-CL,8:2Fluorotelomersulfonic Acid (Report),NaN,<9.40,ppt_wt_v,...,10.0,Metal Finisher B,NaN,Metal Finishing,NaN,NaN,NaN,NaN,NaN,NaN


### Import Other Data

In [7]:
# Import PFAS Attributes

#Specify where the file is saved use '/' instead of '\'
excel_file_path = 'U:/Research_and_Innovation/_ModellingGroupProjects/PFAS Investigations/Data and analysis/Old Databases and Spreadsheets/Database as of 03-25-2024.xlsm'
#Specify sheet name
sheet_name = 'PFAS Attributes'

df_PFAS_Attributes = pd.read_excel(excel_file_path, sheet_name=sheet_name)

In [8]:
# Create a dictionary mapping PFAS Compounds to their corresponding Abbreviation (No need to change)
pfas_mapping = dict(zip(df_PFAS_Attributes['Database'], df_PFAS_Attributes['Abbrev']))

In [9]:
# Load the Excel file into a DataFrame
df = pd.read_excel("U:/Research_and_Innovation/_ModellingGroupProjects/PFAS Investigations/Data and analysis/PFAS Toolbox_ColorsPalette.xlsx",sheet_name='ColorPalette_NonTOP')

def parse_rgb(rgb_string):
    # Check if rgb_string is a string and not empty
    if isinstance(rgb_string, str) and rgb_string:
        # Remove parentheses and split the string by commas
        rgb_values = rgb_string.strip('()').split(',')
        # Convert each value to float and return as a tuple
        return tuple(int(value)/255 for value in rgb_values)
    else:
        # Handle the case where rgb_string is not a string or is empty
        return None  # or return (0, 0, 0, 0) for a default transparent color, for example



# Create a dictionary with PFAS compounds as keys and corresponding colors as values
compound_colors_dict = {compound: parse_rgb(rgb) for compound, rgb in zip(df['Database'], df['New_RGB'])}
compound_colors_dict_abbrev = {abbrev: parse_rgb(rgb) for abbrev, rgb in zip(df['Abbrev'], df['New_RGB'])}

### Data Records for Sampling Location

#### Functions

In [10]:
def create_custom_pivot_table(data_df, value_column='Quant Incl Estimates Incl Qual Flags', filter_qual_summary=None, start_date=None, end_date=None, filter_pfas_compounds=None, filter_sample_location=None):
    """
    Create a custom pivot table with filters based on 'Qual Summary (Grade for filtering)', 'Sample Location',
    'Sample Date', 'PFAS Compounds', and 'Sample Location'.

    Args:
        data_df (pd.DataFrame): The DataFrame containing the data.
        value_column (str, optional): The column to use as pivot table values (default is 'Quant Incl Estimates Incl Qual Flags').
        filter_qual_summary (list of str, optional): List of 'Qual Summary (Grade for filtering)' values to filter the data (default is None).
        start_date (str, optional): Start date for the date range filter (default is None).
        end_date (str, optional): End date for the date range filter (default is None).
        filter_pfas_compounds (list of str, optional): List of 'PFAS Compounds' values to filter the data (default is None).
        filter_sample_location (list of str, optional): List of 'Sample Location' values to filter the data (default is None).

    Returns:
        pd.DataFrame: The custom pivot table with average values.
    """
    try:
        # Apply filters based on 'Qual Summary (Grade for filtering)'
        filtered_data = data_df
        if filter_qual_summary:
            filtered_data = filtered_data[filtered_data['Qual Summary (Grade for filtering)'].isin(filter_qual_summary)]

        # Apply date range filter based on 'Sample Date'
        if start_date and end_date:
            filtered_data = filtered_data[(filtered_data['Sample Date'] >= start_date) & (filtered_data['Sample Date'] <= end_date)]

        # Apply filters based on 'PFAS Compounds'
        if filter_pfas_compounds:
            filtered_data = filtered_data[filtered_data['PFAS Compound'].isin(filter_pfas_compounds)]

        # Apply filter based on 'Sample Location'
        if filter_sample_location:
            filtered_data = filtered_data[filtered_data['Sample Location'].isin(filter_sample_location)]

        # Create a pivot table with 'Sample Date' as columns, 'PFAS Compound' as rows, and specified value column as values
        pivot_table = pd.pivot_table(filtered_data, values=value_column,
                                     index='PFAS Compound', columns='Sample Date',
                                     aggfunc='mean', fill_value=np.nan)

        return pivot_table

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

In [11]:
def create_combined_pivot(Quat, QuatInc, MDL):
    # Create an empty DataFrame with the same structure as the input tables
    combined_pivot = pd.DataFrame(index=Quat.index, columns=Quat.columns)
    
    # Iterate through rows and columns
    for row in Quat.index:
        for col in Quat.columns:
            Quat_value = Quat.loc[row, col] if row in Quat.index and col in Quat.columns else np.nan
            QuatInc_value = QuatInc.loc[row, col] if row in QuatInc.index and col in QuatInc.columns else np.nan
            MDL_value = MDL.loc[row, col] if row in MDL.index and col in MDL.columns else np.nan
            
            # Check if the values are numeric
            if pd.notna(Quat_value):
                Quat_value = float(Quat_value)  # Convert to float
                QuatInc_value = float(QuatInc_value)  # Convert to float
                MDL_value = float(MDL_value)  # Convert to float
                
                if Quat_value > 0:
                    combined_pivot.loc[row, col] = str(Quat_value)
                elif QuatInc_value > 0:
                    combined_pivot.loc[row, col] = str(QuatInc_value) + " E"
                else:
                    combined_pivot.loc[row, col] = "< " + str(MDL_value)
            else:
                combined_pivot.loc[row, col] = ""
    
    return combined_pivot

### Leupold & Stevens PFAS Report

#### Filters and Settings

In [12]:
# Filter criteria
#filter_qual_summary (list of str, optional): List of 'Qual Summary (Grade for filtering)' values to filter the data (default is None).
filter_qual_summary = ['Good','Probably Ok','']
# filter_sample_location (list of str, optional): List of 'Sample Location' values to filter the data (default is None).
filter_sample_location=['Leupold & Stevens Inc']
#start_date='2022-04-12'
#end_date='2024-12-30'

#### Pivot Table for Average of Quant Only Incl Qual Flags

In [13]:
quant_pivot=create_custom_pivot_table(data_df,value_column='Quant Only Incl Qual Flags', filter_qual_summary=filter_qual_summary, filter_pfas_compounds=None, filter_sample_location=filter_sample_location)
quant_pivot

Sample Date,2022-04-13,2022-08-08,2023-01-26,2023-04-06,2023-09-12
PFAS Compound,,,,,
10:2Fluorotelomersulfonic acid,0.0,0.0,0,NaN,0.0
11Cl-PF3OUdS,0.0,0.0,0,NaN,0.0
4:2 Fluorotelomersulfonic acid,0.0,0.0,0,0.0,0.0
6:2 Fluorotelomersulfonic acid,0.0,NaN,21,NaN,37.0
8:2 Fluorotelomersulfonic acid,0.0,0.0,0,0.0,0.0
9Cl-PF3ONS,0.0,0.0,0,NaN,0.0
ADONA,0.0,0.0,0,NaN,0.0
HFPO-DA,NaN,0.0,0,0.0,0.0
NEtFOSA,0.0,0.0,0,NaN,0.0


#### Pivot Table for Quant Incl Estimates Incl Qual Flags

In [14]:
QuatInc_Pivot = create_custom_pivot_table(data_df,value_column='Quant Incl Estimates Incl Qual Flags', filter_qual_summary=filter_qual_summary, filter_pfas_compounds=None, filter_sample_location=filter_sample_location)
QuatInc_Pivot

Sample Date,2022-04-13,2022-08-08,2023-01-26,2023-04-06,2023-09-12
PFAS Compound,,,,,
10:2Fluorotelomersulfonic acid,0.0,0.0,0.0,NaN,0.0
11Cl-PF3OUdS,0.0,0.0,0.0,NaN,0.0
4:2 Fluorotelomersulfonic acid,0.0,0.0,0.0,0.0,0.0
6:2 Fluorotelomersulfonic acid,0.0,NaN,21.0,NaN,37.0
8:2 Fluorotelomersulfonic acid,0.0,0.0,0.0,0.0,0.0
9Cl-PF3ONS,0.0,0.0,0.0,NaN,0.0
ADONA,0.0,0.0,0.0,NaN,0.0
HFPO-DA,NaN,0.0,0.0,0.0,0.0
NEtFOSA,0.0,0.0,0.0,NaN,0.0


#### Pivot Table for Avg MDL

In [15]:
MDL_Pivot=create_custom_pivot_table(data_df,value_column='MDL', filter_qual_summary=filter_qual_summary,  filter_pfas_compounds=None, filter_sample_location=filter_sample_location)
MDL_Pivot

Sample Date,2022-04-13,2022-08-08,2023-01-26,2023-04-06,2023-09-12
PFAS Compound,,,,,
10:2Fluorotelomersulfonic acid,0.92,10.0,8.0,NaN,8.0
11Cl-PF3OUdS,0.46,5.0,5.0,NaN,5.0
4:2 Fluorotelomersulfonic acid,0.46,5.0,5.0,0.77,5.0
6:2 Fluorotelomersulfonic acid,1.80,NaN,NaN,NaN,NaN
8:2 Fluorotelomersulfonic acid,0.92,10.0,6.0,0.92,6.0
9Cl-PF3ONS,0.46,5.0,5.0,NaN,5.0
ADONA,0.46,5.0,5.0,NaN,5.0
HFPO-DA,NaN,10.0,4.0,0.62,4.0
NEtFOSA,0.92,10.0,4.0,NaN,4.0


#### Data Copillation Table

In [16]:
# Call function and provide previous tables as inputs
PFAS_Results_Location = create_combined_pivot(Quat=quant_pivot, QuatInc=QuatInc_Pivot, MDL=MDL_Pivot)

# Create a dictionary mapping PFAS Compounds to their corresponding Abbreviation (No need to change)
pfas_mapping = dict(zip(df_PFAS_Attributes['Database'], df_PFAS_Attributes['Abbrev']))

# Rename the index of data_df using the mapping (No need to change)
PFAS_Results_Location.index = PFAS_Results_Location.index.map(pfas_mapping)

PFAS_Results_Location

Sample Date,2022-04-13,2022-08-08,2023-01-26,2023-04-06,2023-09-12
PFAS Compound,,,,,
10:2FTS,< 0.92,< 10.0,< 8.0,,< 8.0
11Cl-PF3OUdS,< 0.46,< 5.0,< 5.0,,< 5.0
4:2FTS,< 0.46,< 5.0,< 5.0,< 0.77,< 5.0
6:2FTS,< 1.8,,21.0,,37.0
8:2FTS,< 0.92,< 10.0,< 6.0,< 0.92,< 6.0
9Cl-PF3ONS,< 0.46,< 5.0,< 5.0,,< 5.0
ADONA,< 0.46,< 5.0,< 5.0,,< 5.0
HFPO-DA,,< 10.0,< 4.0,< 0.62,< 4.0
NEtFOSA,< 0.92,< 10.0,< 4.0,,< 4.0


#### Export Data Records

In [17]:
#dfs: A dictionary where keys are the tab names and values are the DataFrames to write.
dfs = {'Quant': quant_pivot,
       'QuantIncEstimate' :QuatInc_Pivot,
       'MDLs' : MDL_Pivot,
       'PFAS Report' : PFAS_Results_Location}

#filename: The name of the Excel file to write to.
filename= 'Leupold & Stevens_PFAS Report.xlsx'
dfs_to_excel(dfs, filename)

File 'Leupold & Stevens_PFAS Report.xlsx' has been written with 4 tabs.
